# Chapter XX -- Quantum Information

In this chapter we will discuss core concepts in quantum information.  Core principles are explained through the lens of photonic implementations of quantum infromation as this will be how we explore these concepts in the laboratory for this course.  However, you should note that these concepts extend to any quantum information system.  

This recitation will specifically cover the following topics:

1. Quantum Bits (qubits)
2. Quantum Gates
3. Quantum Teleportation
4. Quantum Error Correction
5. Quantum Cryptography
6. Quantum Key Distribution

## Packages, Functions and Variables

The following code-block contains the packages, functions and variables needed for the code found later in this notebook.  It must be run first.  Note that if you are running this on your personal computer, you should ensure that all packages needed are installed first (for example the [numpy](https://numpy.org/), [scipy](https://scipy.org/), and [matplotlib](https://matplotlib.org/) packages must be installed on your computer along with others).  You can find a launcher link at the top to conveniently run this notebook in Google Colab.  

The code below is hidden by default, but you are encouraged to expand it and study it.  The functions and constants will update as needed during the progression of the course, so you should note what new functions have been and added and study how they work.  

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets
import h5py
from scipy import signal
from scipy.interpolate import UnivariateSpline 

# Import the matplotlib animation tools
# as well as the html tools for creating
# javascript animations
from matplotlib import animation, rc
from IPython.display import HTML

#-- Export Settings --
# If on, then it applies a few tweaks to make sure the export looks good
want_export=False

#--Physical Constants--
hbar = 1.054e-34 #normalized Planck constant [J*s]
evcon = 1.602e-19 #conversion factor, [J/eV]
eta = 119.91699832*np.pi #impedance of freespace [ohms]
me = 9.10938188e-31 #mass of electron [kg]
c = 2.99792458e8 #speed of light [m/s]
e = 1.602e-19 #electron charge [C]

## XX.1 Quantum Bits (qubits)

Analogous to the bit, the fundamental unit of information of classical computers, a quantum bit (qubit)
corresponds to a two level system. However, the qubit can have both values at the same time:

$\begin{equation}
\ket{\psi} = c_0 \ket{0} + c_1 \ket{1} 
\end{equation}$

where $c_i$ are complex numbers and $\sum_i c_i^2 = 1$. Physically, this state could be represented by the polarization
of a photon in a particular basis (e.g., linear or circular), the population of a photon in a particular mode, or the spin orientation of a spin-1/2 particle such as an electron or spin-1/2 nucleus.

Now let's consider two classical bits, which have four possible states: 

$\begin{equation} \lbrace 00, 01, 10, 11 \rbrace \end{equation}$

The quantum register consisting of two qubits is again in all four possible states at once:

$\begin{equation} \ket{\psi}  = c_{00}\ket{00} + c_{01}\ket{01} + c_{10}\ket{10} + c_{11}\ket{11} \end{equation}$

This simple example already shows one of the fundamental differences -- and potential strengths -- of the quantum register: it is in all states at once.  

For instance, suppose we wanted to represent the state of $N$ two-level particles, such as the polarization states of photons or the orientation of electron spins. This would us to store $2^N$ complex numbers, i.e., $2^{N+1}$ real numbers. Suppose each real number were stored in a 32-bit classical register. Then we'd need $32\times2^{N+1}$ bits. If $N=50$, that's already 4500 tera-bytes, and if $N=500$, we would require more bits than there are atoms in the universe (double-check!).  In a quantum computer, a register of 500 qubits would suffice! Thus, we already see that for certain applications -- like trying to simulate a quantum system -- the quantum memory is exponentially more powerful than a classical memory. The idea of using a quantum system for simulating a classical system was proposed by Richard Feynman in the early eighties, and is called quantum simulation. 

Going back to our two-level system, an important set of state are the Bell states:

$\begin{equation} \bigg \lbrace \frac{1}{\sqrt{2}} \big ( \ket{00} \pm \ket{11} \big ), \frac{1}{\sqrt{2}} \big (\ket{10} \pm \ket{01} \big ) \bigg \rbrace \end{equation}$

These are maximally entangled states. Let's consider one of these in more detail, say:

$\begin{equation} \ket{\beta_{00}} = \frac{1}{\sqrt{2}} \big ( \ket{00} + \ket{11} \big ) \end{equation}$

If we measure the first qubit, we obtain either '0' or '1' with equal probability. If we measure '1', then the state is projected to $\ket{\psi} = \ket{11}$; if we measure '0', then the state is projected to $\ket{\psi} = \ket{00}$. The measurement outcomes are correlated, even though the two qubits could be a different locations. 

### XX.1.1 Quantum Gates

#### Single Qubit Gates

1. NOT Gate

   $X = \ket{1} \bra{0} + \ket{0}\bra{1}$
 
   $X\ket{\psi} = X(c_0 \ket{0} + c_1 \ket{1}) = c_0 \ket{1} + c_1 \ket{0}$
   
   Photonic implementation: e.g., a half wave plate (HWP) at 45 degrees to the linear polarization basis.  Recall the Jones matrix for a HWP:
   
   $\begin{equation}
   M_{\text{HWP}} = 
   \begin{pmatrix}
    \cos{2\theta} & \sin{2\theta} \\
    \sin{2\theta} & -\cos{2\theta}
   \end{pmatrix}
   \end{equation}$
   
2. Z Gate

   $Z = \ket{0} \bra{0} - \ket{1}\bra{1}$
   
   $Z\ket{\psi} = Z(c_0 \ket{0} + c_1 \ket{1}) = c_0 \ket{0} - c_1 \ket{1}$
   
   Photonic implementation: e.g., a HWP with $\theta = 0$.
   
3. Hadamard Gate

   The purpose of this gate is to transform from the input basis to the complimentary basis.  In terms of a polarization basis, it would be essentially a rotation by 45 degrees.  

   $H = \ket{0} \bra{0} + \ket{1}\bra{0} + \ket{0}\bra{1} - \ket{1}\bra{1}$
   
   $H\ket{\psi} = H(c_0 \ket{0} + c_1 \ket{1}) = \frac{1}{\sqrt{2}} \big ( c_0 \ket{0} + c_0 \ket{1} + c_1\ket{0} - c_1\ket{1} \big )$
   
   $\begin{equation}
   M_{\text{H}} = 
   \frac{1}{\sqrt{2}} 
   \begin{pmatrix}
    1 & 1 \\
    1 & -1
   \end{pmatrix}
   \end{equation}$
   
   Photonic implementation: for polarization states, a HWP at $\theta = 22.5^{\circ}$; for mode basis states, a beamsplitter.  



#### Two Qubit Gates

### System Settings

Interactive script for easily inputting all desired settings for the actively mode-locked cavity.  All variable names derived from the related equations introduced above. 